# Array node layouts

This document describes, in a minimal way, each of the composable array layout nodes that collectively describe structured data in a columnar way.

All of the constraints on allowed values in an Awkward Array are expressed here as assertions in `__init__`. The `random` constructors create a random valid array. The data structure is defined by its length `__len__`, its `__getitem__` behavior for integers, slices (without step), and string fields. Iteration with `__iter__` converts the columnar data into rowwise data.

## General structure

Arrays are composed of `Content` subclasses and integer arrays. Integer arrays (called `Indexes`) determine the structure, but not the content of the data structure.

In [ ]:
class Content:
    def __iter__(self):
        "Iterate over the data structure, converting columnar data into rowwise."
        
        def convert(x):
            if isinstance(x, Content):
                return list(x)
            elif isinstance(x, tuple):
                return tuple(convert(y) for y in x)
            elif isinstance(x, dict):
                return {n: convert(y) for n, y in x.items()}
            else:
                return x

        for i in range(len(self)):
            yield convert(self[i])

    def __repr__(self):
        return self.tostring_part("", "", "").rstrip()

    @staticmethod
    def random(minlen=0, choices=None):
        if choices is None:
            choices = [x for x in globals().values() if isinstance(x, type) and issubclass(x, Content)]
        else:
            choices = list(choices)
        if minlen != 0 and EmptyArray in choices:
            choices.remove(EmptyArray)
        assert len(choices) > 0
        cls = random.choice(choices)
        return cls.random(minlen, choices)

In [ ]:
def random_number():
    return round(random.gauss(5, 3), 1)

def random_length(minlen=0, maxlen=None):
    if maxlen is None:
        return minlen + int(math.floor(random.expovariate(0.1)))
    else:
        return random.randint(minlen, maxlen)